# **Instruções de Uso:** Clique em "Ambiente de execução" depois em "Executar tudo" (CTRL + F9), e aguarde até que a interface do Gradio apareça.

# Na interface, selecione a aba que deseja utilizar, gerar o modelo através de uma imagem, ou através de um prompt.

# SETUP

In [ ]:
!pip install torch torchvision
!pip install onnxruntime
!pip install rembg
!pip install trimesh
!pip install -q gradio

In [ ]:
!git clone https://github.com/tencent/Hunyuan3D-2.git

In [ ]:
%cd Hunyuan3D-2
!pip install -r requirements.txt
!cd hy3dgen/texgen/custom_rasterizer && pip install .
!cd hy3dgen/texgen/differentiable_renderer && bash compile_mesh_painter.sh

In [ ]:
!pip install diffusers==0.29.1

In [ ]:
from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline

pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained('tencent/Hunyuan3D-2')

# GERAR MODELO 3D COM BASE EM UMA IMAGEM EXISTENTE

In [ ]:
from rembg import remove
from PIL import Image
from google.colab import files

def generate_stl_from_image(img, format=".glb"):

    if isinstance(img, str):
        img = Image.open(img)
        img = np.array(img)

    img = Image.fromarray(img)
    img_sem_fundo = remove(img)

    mesh = pipeline(image=img_sem_fundo)[0]
    file_path_visu = "output_image_visu.glb"
    file_path = "output_image"+format

    # Export file to visualization
    mesh.export(file_path_visu)
    # Export file to download
    mesh.export(file_path)

    return file_path_visu, file_path

# GERAR MODELO 3D COM BASE NUM PROMPT

In [ ]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management
from rembg import remove

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2


In [ ]:
def generate_image(prompt, format=".glb"):
  with torch.inference_mode():
    positive_prompt = prompt
    width = 720
    height = 720
    seed = 0
    steps = 20
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")

  imagem = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

  img_sem_fundo = remove(imagem)

  mesh = pipeline(image=img_sem_fundo)[0]
  file_path_visu = "output_image_visu.glb"
  file_path = "output_image"+format

  # Export file to visualization
  mesh.export(file_path_visu)
  # Export file to download
  mesh.export(file_path)

  return file_path_visu, file_path

# Image Style

In [ ]:
!git clone https://github.com/TencentARC/PhotoMaker.git

In [ ]:
%cd PhotoMaker
!pip install -r requirements.txt
!pip install git+https://github.com/TencentARC/PhotoMaker.git
%cd ..

In [ ]:
import torchvision.transforms.functional as TF
import random
import os

from diffusers.utils import load_image
from diffusers import EulerDiscreteScheduler, T2IAdapter

from huggingface_hub import hf_hub_download
import spaces
import gradio as gr

from photomaker import PhotoMakerStableDiffusionXLAdapterPipeline
from photomaker import FaceAnalysis2, analyze_faces
import torch

# global variable
base_model_path = 'SG161222/RealVisXL_V4.0'
face_detector = FaceAnalysis2(providers=['CUDAExecutionProvider'], allowed_modules=['detection', 'recognition'])
face_detector.prepare(ctx_id=0, det_size=(640, 640))

try:
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
except:
    device = "cpu"


DEFAULT_STYLE_NAME = "Photographic (Default)"


enable_doodle_arg = False
photomaker_ckpt = hf_hub_download(repo_id="TencentARC/PhotoMaker-V2", filename="photomaker-v2.bin", repo_type="model")

torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
if device == "mps":
    torch_dtype = torch.float16

# load adapter
adapter = T2IAdapter.from_pretrained(
    "TencentARC/t2i-adapter-sketch-sdxl-1.0", torch_dtype=torch_dtype, variant="fp16"
).to(device)

pipe = PhotoMakerStableDiffusionXLAdapterPipeline.from_pretrained(
    base_model_path,
    adapter=adapter,
    torch_dtype=torch_dtype,
    use_safetensors=True,
    variant="fp16",
).to(device)

pipe.load_photomaker_adapter(
    os.path.dirname(photomaker_ckpt),
    subfolder="",
    weight_name=os.path.basename(photomaker_ckpt),
    trigger_word="img",
    pm_version="v2",
)
pipe.id_encoder.to(device)

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.set_adapters(["photomaker"], adapter_weights=[1.0])
pipe.fuse_lora()
pipe.to(device)


In [ ]:
#Define styles:
style_list = [
    {
        "name": "(No style)",
        "prompt": "{prompt}",
        "negative_prompt": "",
    },
    {
        "name": "Cinematic",
        "prompt": "cinematic still {prompt} . emotional, harmonious, vignette, highly detailed, high budget, bokeh, cinemascope, moody, epic, gorgeous, film grain, grainy",
        "negative_prompt": "anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured",
    },
    {
        "name": "Disney Charactor",
        "prompt": "A Pixar animation character of {prompt} . pixar-style, studio anime, Disney, high-quality",
        "negative_prompt": "lowres, bad anatomy, bad hands, text, bad eyes, bad arms, bad legs, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, blurry, grayscale, noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo",
    },
    {
        "name": "Digital Art",
        "prompt": "concept art {prompt} . digital artwork, illustrative, painterly, matte painting, highly detailed",
        "negative_prompt": "photo, photorealistic, realism, ugly",
    },
    {
        "name": "Photographic (Default)",
        "prompt": "cinematic photo {prompt} . 35mm photograph, film, bokeh, professional, 4k, highly detailed",
        "negative_prompt": "drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly",
    },
    {
        "name": "Fantasy art",
        "prompt": "ethereal fantasy concept art of  {prompt} . magnificent, celestial, ethereal, painterly, epic, majestic, magical, fantasy art, cover art, dreamy",
        "negative_prompt": "photographic, realistic, realism, 35mm film, dslr, cropped, frame, text, deformed, glitch, noise, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, sloppy, duplicate, mutated, black and white",
    },
    {
        "name": "Neonpunk",
        "prompt": "neonpunk style {prompt} . cyberpunk, vaporwave, neon, vibes, vibrant, stunningly beautiful, crisp, detailed, sleek, ultramodern, magenta highlights, dark purple shadows, high contrast, cinematic, ultra detailed, intricate, professional",
        "negative_prompt": "painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured",
    },
    {
        "name": "Enhance",
        "prompt": "breathtaking {prompt} . award-winning, professional, highly detailed",
        "negative_prompt": "ugly, deformed, noisy, blurry, distorted, grainy",
    },
    {
        "name": "Comic book",
        "prompt": "comic {prompt} . graphic illustration, comic art, graphic novel art, vibrant, highly detailed",
        "negative_prompt": "photograph, deformed, glitch, noisy, realistic, stock photo",
    },
    {
        "name": "Lowpoly",
        "prompt": "low-poly style {prompt} . low-poly game art, polygon mesh, jagged, blocky, wireframe edges, centered composition",
        "negative_prompt": "noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo",
    },
    {
        "name": "Line art",
        "prompt": "line art drawing {prompt} . professional, sleek, modern, minimalist, graphic, line art, vector graphics",
        "negative_prompt": "anime, photorealistic, 35mm film, deformed, glitch, blurry, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, mutated, realism, realistic, impressionism, expressionism, oil, acrylic",
    }
]

styles = {k["name"]: (k["prompt"], k["negative_prompt"]) for k in style_list}

In [ ]:
import gradio as gr

aspect_ratios = [640, 1024]

DEFAULT_STYLE_NAME = "Photographic (Default)"

def apply_style(style_name: str, positive: str, negative: str = ""):
    p, n = styles.get(style_name, styles[DEFAULT_STYLE_NAME])
    return p.replace("{prompt}", positive), n + ' ' + negative

def generate_style_image_v2(
        upload_images,
        prompt,
        negative_prompt,
        style_name,
        num_steps,
        style_strength_ratio,
        num_outputs,
        guidance_scale,
        seed,
        progress=gr.Progress(track_tqdm=True)):

    # check the trigger word
    image_token_id = pipe.tokenizer.convert_tokens_to_ids(pipe.trigger_word)
    # prompt = prompt + ", face only portrait,"

    input_ids = pipe.tokenizer.encode(prompt)
    if image_token_id not in input_ids:
        raise gr.Error(f"Cannot find the trigger word '{pipe.trigger_word}' in text prompt!")

    if input_ids.count(image_token_id) > 1:
        raise gr.Error(f"Cannot use multiple trigger words '{pipe.trigger_word}' in text prompt!")

    # determine output dimensions by the aspect ratio
    aspect_ratio_name="35mm film / Portrait (2:3)"
    output_w, output_h = aspect_ratios
    print(f"[Debug] Generate image using aspect ratio [{aspect_ratio_name}] => {output_w} x {output_h}")

    # apply the style template
    prompt, negative_prompt = apply_style(style_name, prompt, negative_prompt)

    print(f"[DEBUG] Image path: {upload_images}")

    if upload_images is None:
        raise gr.Error(f"Cannot find any input face image!")

    input_id_images = []
    for img in upload_images:
        input_id_images.append(load_image(img))

    # input_id_images.append(load_image(upload_images))

    id_embed_list = []

    for img in input_id_images:
        img = np.array(img)
        img = img[:, :, ::-1]
        faces = analyze_faces(face_detector, img)
        if len(faces) > 0:
            id_embed_list.append(torch.from_numpy((faces[0]['embedding'])))

    if len(id_embed_list) == 0:
        raise gr.Error(f"No face detected, please update the input face image(s)")

    id_embeds = torch.stack(id_embed_list)

    generator = torch.Generator(device=device).manual_seed(222)

    print("Start inference...")
    print(f"[Debug] Seed: {seed}")
    print(f"[Debug] Prompt: {prompt}, \n[Debug] Neg Prompt: {negative_prompt}")
    start_merge_step = int(float(style_strength_ratio) / 100 * num_steps)
    if start_merge_step > 15:
        start_merge_step = 15
    print(start_merge_step)
    images = pipe(
        prompt=prompt,
        width=output_w,
        height=output_h,
        input_id_images=input_id_images,
        negative_prompt=negative_prompt,
        num_images_per_prompt=1,
        num_inference_steps=num_steps,
        start_merge_step=start_merge_step,
        generator=generator,
        guidance_scale=guidance_scale,
        id_embeds=id_embeds,
        image=None,
        adapter_conditioning_scale=0,
        adapter_conditioning_factor=0,
    ).images
    return images[0]

# MAIN

In [ ]:
formats_list = [".stl", ".glb", ".obj"]

# Interface Gradio
def swap_to_gallery(images):
    return gr.update(value=images, visible=True),  gr.update(visible=False)

with gr.Blocks() as iface:
    gr.Markdown("# Selecionar Método de Geração")
    with gr.Tabs():

      with gr.Tab("Geração por Imagem"):
          # Entrada de imagem original
          with gr.Row():
              # input_image = gr.Image(label="Imagem de Entrada", type="filepath")
              files = gr.Files(
                    label="Imagem de Entrada",
                    file_types=["image"]
                )
              uploaded_files = gr.Gallery(label="Sua imagem", visible=False, columns=5, rows=1, height=200)

              # Coluna de estilização
              with gr.Column():
                  gr.Markdown("### Opções de Estilização")

                  # Seleção de estilo
                  style_selector = gr.Radio(
                      choices=[style["name"] for style in style_list],
                      label="Selecione o Estilo",
                      value=DEFAULT_STYLE_NAME
                  )

                  # Parâmetros de geração
                  with gr.Row():
                      num_steps = gr.Slider(minimum=15, maximum=100, value=50, label="Número de Passos")
                      style_strength = gr.Slider(minimum=0, maximum=100, value=20, label="Força do Estilo (%)")

                  # Outros parâmetros
                  guidance_scale = gr.Slider(minimum=1, maximum=20, value=5, label="Guidance Scale")
                  seed = gr.Number(value=0, label="Semente Aleatória")

                  # Prompt opcional para estilização
                  style_prompt = gr.Textbox(
                      label="Prompt Opcional para Estilização",
                      placeholder="Descreva modificações desejadas",
                      lines=2
                  )

          # Botão para aplicar estilização
          btn_stylize = gr.Button("Aplicar Estilização")

          # Saída da imagem estilizada
          #styled_image_output = gr.Image(label="Imagem Estilizada", type="filepath")
          styled_image_output = gr.Image(label="Imagem Estilizada", visible=False)

          files.upload(fn=swap_to_gallery, inputs=files, outputs=[uploaded_files, files])

          with gr.Column():
              gallery = gr.Image(label="Imagem Estilizada", interactive=False)

          # Fluxo de eventos
          # 1. Aplicar estilização
          btn_stylize.click(
              generate_style_image_v2,
              inputs=[
                  files,                       # upload_images (first)
                  style_prompt,                      # prompt
                  gr.Textbox(value="", visible=False), # negative_prompt
                  style_selector,                    # style_name
                  num_steps,                         # num_steps
                  style_strength,                    # style_strength_ratio
                  gr.Number(value=1),                # num_outputs
                  guidance_scale,                    # guidance_scale
                  seed                               # seed
              ],
              outputs=[styled_image_output]
          )

          styled_image_output.change(
              lambda img: img if img is not None else None,
              inputs=styled_image_output,
              outputs=gallery
          )


          use_styled = gr.Radio(choices=["Imagem Estilizada", "Imagem Original"],
                                label="Selecione a imagem para gerar o mesh",
                                value="Imagem Original")


          mesh_output = gr.Model3D(clear_color=[0.0, 0.0, 0.0, 0.0], label="Visualização 3D")
          download_file = gr.File(label="Download do Arquivo 3D")


          def generate_mesh(use_styled_val, format_selector, styled_img, original_files):
              if use_styled_val == "Imagem Estilizada" and styled_img is not None:
                  img_to_use = styled_img
              else:
                  img_to_use = original_files[0] if original_files else None
              if img_to_use is None:
                  return "Nenhuma imagem selecionada."

              return generate_stl_from_image(img_to_use, format_selector)

          format_selector = gr.Radio(
              choices=[format for format in formats_list],
              label="Formato de Saída",
              value=formats_list[0]
          )

          btn_generate_mesh = gr.Button("Gerar Mesh")
          btn_generate_mesh.click(
              generate_mesh,
              inputs=[use_styled, format_selector, styled_image_output, files],
              outputs=[mesh_output, download_file]
          )


      with gr.Tab("Gerar por Prompt"):
          prompt_input = gr.Textbox(label="Digite o Prompt")
          # prompt_output = gr.File(label="Download do STL")

          mesh_prompt_output = gr.Model3D(clear_color=[0.0, 0.0, 0.0, 0.0], label="Visualização 3D")
          download_prompt_file = gr.File(label="Download da Mesh")

          btn_prompt = gr.Button("Gerar Imagem")

          format_selector_prompt = gr.Radio(
              choices=[format for format in formats_list],
              label="Formato de Saída",
              value=formats_list[0]
          )
          btn_prompt.click(generate_image, inputs=[prompt_input, format_selector_prompt], outputs=[mesh_prompt_output, download_prompt_file])

# Lança a interface
iface.launch(debug=True)